## Polytope Climate-DT Time Series example notebook for story nudging

This notebook shows how to use earthkit-data and earthkit-plots to pull destination-earth data from LUMI and plot it using earthkit-plots.

Before running the notebook you need to set up your credentials. See the main readme of this repository for different ways to do this or use the following cells to authenticate.

You will need to generate your credentials using the desp-authentication.py script.

This can be run as follows:

In [14]:
%%capture cap
%run ../desp-authentication.py

This will generate a token that can then be used by earthkit and polytope.

In [15]:
output_1 = cap.stdout.split('}\n')
access_token = output_1[-1][0:-1]

# Requirements
To run this notebook install the following:
* pip install earthkit-data
* pip install earthkit-plots
* pip install earthkit-regrid  (Optional for spectral variables)
* pip install cf-units         (Optional for unit conversion in maps)

If you do not have eccodes installed please install eccodes using conda as it is a dependency of earthkit, or install earthkit via conda

* conda install eccodes -c conda-forge
* conda install earthkit-data -c conda-forge

In [4]:
import earthkit.data
from earthkit.plots.interactive import Chart
from polytope.api import Client

In [17]:
client = Client(
    address="polytope.lumi.apps.dte.destination-earth.eu")

request = {
     "activity": "story-nudging",
     "class": "d1",
     "dataset": "climate-dt",
     "experiment": "cont",
     "generation": "1",
     "levtype": "sfc",
     "date": "20180205/to/20180220",
     "model": "ifs-fesom",
     "expver": "0001",
     "param": "165",
     "realization": "1",
     "resolution": "high",
     "stream": "clte",
     "type": "fc",
     "time": "0000",
     "feature": {
         "type": "timeseries",
         "points": [[38, -9.5]],
         "time_axis": "date",
     }
 }

file = client.retrieve("destination-earth", request, "data/climate-dt-earthkit-fe-story-nudging.covjson") 

In [11]:
data = earthkit.data.from_source("file", "data/climate-dt-earthkit-fe-story-nudging.covjson") 

In [12]:
data._json()

{'type': 'CoverageCollection',
 'domainType': 'PointSeries',
 'coverages': [{'mars:metadata': {'activity': 'story-nudging',
    'class': 'd1',
    'dataset': 'climate-dt',
    'experiment': 'cont',
    'expver': '0001',
    'generation': '1',
    'levtype': 'sfc',
    'model': 'ifs-fesom',
    'realization': '1',
    'resolution': 'high',
    'stream': 'clte',
    'type': 'fc',
    'number': 0,
    'Forecast date': '2018-02-20 00:00:00Z'},
   'type': 'Coverage',
   'domain': {'type': 'Domain',
    'axes': {'latitude': {'values': [38.016284446058]},
     'longitude': {'values': [350.5078125]},
     'levelist': {'values': [0]},
     't': {'values': ['2018-02-05 00:00:00Z',
       '2018-02-06 00:00:00Z',
       '2018-02-07 00:00:00Z',
       '2018-02-08 00:00:00Z',
       '2018-02-09 00:00:00Z',
       '2018-02-10 00:00:00Z',
       '2018-02-11 00:00:00Z',
       '2018-02-12 00:00:00Z',
       '2018-02-13 00:00:00Z',
       '2018-02-14 00:00:00Z',
       '2018-02-15 00:00:00Z',
       '20

In [14]:
def location_to_string(location):
    """
    Converts latitude and longitude to a string representation with degrees
    and N/S/E/W.
    """
    (lat, lon) = location
    lat_dir = "N" if lat >= 0 else "S"
    lon_dir = "E" if lon >= 0 else "W"
    return f"{abs(lat):.2f}°{lat_dir}, {abs(lon):.2f}°{lon_dir}"

In [13]:
ds = data.to_xarray()

In [16]:
TIME_FREQUENCY = "1d"
QUANTILES = [0, 0.1, 0.25, 0.5, 0.75, 0.9, 1]
LOCATION = ((38, -9.5))

chart = Chart()
chart.title(f"ECMWF ensemble meteogram at {location_to_string(LOCATION)}")
#chart.box(ds, time_frequency=TIME_FREQUENCY, quantiles=QUANTILES)
chart.line(ds, line_color='grey', time_frequency=TIME_FREQUENCY)
chart.show() # Replace with chart.show() in an interactive session!